In [1]:
import numpy as np
import pandas as pd

In [2]:
orders = pd.read_csv('../data/orders.csv')
products = pd.read_csv('../data/products.csv')
order_products__prior = pd.read_csv('../data/order_products__prior.csv')
order_products__train = pd.read_csv('../data/order_products__train.csv')
aisles = pd.read_csv('../data/aisles.csv')
departments = pd.read_csv('../data/departments.csv')

In [3]:
order_products = order_products__prior.append(order_products__train)

In [4]:
full_df = pd.merge(orders, order_products, how='inner', on='order_id')

In [5]:
full_df = pd.merge(full_df, products, how='left', on='product_id')

In [6]:
full_df = pd.merge(full_df, aisles, how='left', on='aisle_id')

In [7]:
full_df = pd.merge(full_df, departments, how='left', on='department_id')

In [8]:
full_df.drop(['order_id', 'eval_set', 'product_id', 'aisle_id', 'department_id', 'add_to_cart_order', 
              'reordered', 'product_name', 'order_number'], axis=1, inplace=True)

In [9]:
full_df.set_index('user_id', inplace=True)

In [10]:
full_df.head()

,order_dow,order_hour_of_day,days_since_prior_order,aisle,department
user_id,,,,,
1,2,8,NaN,soft drinks,beverages
1,2,8,NaN,soy lactosefree,dairy eggs
1,2,8,NaN,popcorn jerky,snacks
1,2,8,NaN,popcorn jerky,snacks
1,2,8,NaN,paper goods,household


In [11]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33819106 entries, 1 to 206209
Data columns (total 5 columns):
order_dow                 int64
order_hour_of_day         int64
days_since_prior_order    float64
aisle                     object
department                object
dtypes: float64(1), int64(2), object(2)
memory usage: 1.5+ GB


In [12]:
full_df.isnull().sum()

order_dow                       0
order_hour_of_day               0
days_since_prior_order    2078068
aisle                           0
department                      0
dtype: int64

In [13]:
full_df.days_since_prior_order.fillna(-1, inplace=True)

In [14]:
full_df.days_since_prior_order = full_df.days_since_prior_order.astype(int)

In [15]:
full_df.days_since_prior_order.unique()

array([-1, 15, 21, 29, 28, 19, 20, 14,  0, 30, 10,  3,  8, 13, 27,  6,  9,
       12,  7, 17, 11,  4,  5,  2, 23, 26, 25, 16,  1, 18, 24, 22])

In [16]:
full_df.loc[full_df.days_since_prior_order == -1, 'days_since_prior_order'] = 'first_order'

In [17]:
full_df.days_since_prior_order.unique()

array(['first_order', 15, 21, 29, 28, 19, 20, 14, 0, 30, 10, 3, 8, 13, 27,
       6, 9, 12, 7, 17, 11, 4, 5, 2, 23, 26, 25, 16, 1, 18, 24, 22], dtype=object)

In [18]:
full_df.info(memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33819106 entries, 1 to 206209
Data columns (total 5 columns):
order_dow                 int64
order_hour_of_day         int64
days_since_prior_order    object
aisle                     object
department                object
dtypes: int64(2), object(3)
memory usage: 1.5+ GB


In [19]:
for col in full_df:
    full_df[col] = full_df[col].astype('category')

In [20]:
full_df.info(memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33819106 entries, 1 to 206209
Data columns (total 5 columns):
order_dow                 category
order_hour_of_day         category
days_since_prior_order    category
aisle                     category
department                category
dtypes: category(5)
memory usage: 451.5 MB


In [21]:
%%time
df_minus_aisles = pd.DataFrame()
for col in ['order_dow', 'order_hour_of_day', 'days_since_prior_order', 'department']:
    df2 = pd.get_dummies(full_df[col], prefix=col)
    df2.reset_index(inplace=True)
    df3 = df2.groupby('user_id').sum()
    df4 = df3.div(df3.sum(axis=1), axis=0)
    df_minus_aisles = pd.concat([df_minus_aisles, df4], axis=1)

CPU times: user 22.5 s, sys: 13.8 s, total: 36.3 s
Wall time: 36.2 s


In [22]:
%%time
df2 = pd.get_dummies(full_df[['aisle']], prefix='aisle')
df3 = df2.groupby('user_id', sort=False).sum()
df_aisles = df3.div(df3.sum(axis=1), axis=0)

CPU times: user 33min 32s, sys: 38.7 s, total: 34min 11s
Wall time: 34min 10s


In [23]:
df = pd.concat([df_minus_aisles, df_aisles], axis=1)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 206209 entries, 1 to 206209
Columns: 218 entries, order_dow_0 to aisle_yogurt
dtypes: float64(218)
memory usage: 344.5 MB


In [25]:
df.head()

,order_dow_0,order_dow_1,order_dow_2,order_dow_3,order_dow_4,order_dow_5,order_dow_6,order_hour_of_day_0,order_hour_of_day_1,order_hour_of_day_2,...,aisle_spreads,aisle_tea,aisle_tofu meat alternatives,aisle_tortillas flat bread,aisle_trail mix snack mix,aisle_trash bags liners,aisle_vitamins supplements,aisle_water seltzer sparkling water,aisle_white wines,aisle_yogurt
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.242857,0.128571,0.157143,0.471429,0.000000,0.000000,0.0,0.0,0.0,...,0.014286,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.028571
2,0.000000,0.420354,0.384956,0.128319,0.039823,0.026549,0.000000,0.0,0.0,0.0,...,0.013274,0.004425,0.004425,0.000000,0.0,0.0,0.0,0.008850,0.0,0.185841
3,0.522727,0.181818,0.056818,0.238636,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.045455,0.011364,0.000000,0.000000,0.0,0.0,0.0,0.022727,0.0,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.500000,0.277778,0.222222,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.055556,0.0,0.0,0.0,0.055556,0.0,0.000000
5,0.391304,0.260870,0.000000,0.347826,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.065217


In [26]:
df.to_csv('../data/df.csv')